<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/L09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum Software Development
# Lab 9: Shor's Factorization Algorithm
# Copyright 2024 The MITRE Corporation. All Rights Reserved.

# Note: Use little endian ordering when storing and retrieving integers from
# qubit registers in this lab.

In [ ]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

In [ ]:
# E01

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    inv_c = pow(c, -1, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

# quantum modular exponentiation

def exp_mod(a, b, input_qubits, output_qubits):
    qc = QuantumCircuit(input_qubits, output_qubits)
    classical_exp_mod = pow(a, int(''.join(str(x) for x in input_qubits)), b)
    modular_multiply_by_constant(qc, b, classical_exp_mod, output_qubits)
    return qc

In [1]:
# E02

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply the exp_mod circuit
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Measure the qubits
    qc.measure(output_qubits, classical_bits)

    return qc

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts(qc)

    # Execute the circuit
    # backend = Aer.get_backend('qasm_simulator')
    # tqc = transpile(qc, backend)
    # qobj = assemble(tqc)
    result = execute(qc, backend).result()

    #counts = result.get_counts()
    print(counts)

In [4]:
# E02

def inverse_qft(qc, qubits):
    qc.append(QFT(len(qubits), inverse=True).to_instruction(), qubits)

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Apply the inverse QFT to input qubits
    inverse_qft(qc, input_qubits)

    # Measure the input qubits
    qc.measure(input_qubits, classical_bits)

    return qc

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts(qc)

    # Execute the circuit
    # backend = Aer.get_backend('qasm_simulator')
    # tqc = transpile(qc, backend)
    # qobj = assemble(tqc)
    result = execute(qc, backend).result()

    #counts = result.get_counts()
    print(counts)

In [2]:
# E03

from sympy import Rational

def find_period_candidate(numerator, denominator, denominator_threshold):
    fraction = Rational(numerator, denominator)
    continued_fraction = fraction.continued_fraction()
    convergents = continued_fraction.convergents()

    for convergent in convergents:
        if convergent.q < denominator_threshold:
            candidate = convergent
        else:
            break
    return (candidate.p, candidate.q)

In [3]:
# E04

def find_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply the exp_mod circuit
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Measure the qubits
    qc.measure(output_qubits, classical_bits)

    return qc

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts(qc)

    # Execute the circuit
    # backend = Aer.get_backend('qasm_simulator')
    # tqc = transpile(qc, backend)
    # qobj = assemble(tqc)
    result = execute(qc, backend).result()

    #counts = result.get_counts()
    print(counts)